In [1]:
import numpy as np
import math
from prettytable import PrettyTable
import random

In [2]:
# Считывание данных из файла
def read_data(file_name):
    f = open(file_name, 'r')
    x = np.zeros((1, 2), dtype=np.int32)
    y = np.zeros(1)
    for line in f:
        data = line.split()
        if (len(data) == 3):
            y = np.append(y, float(data[2]))
            x0 = float(data[0])
            x1 = float(data[1])
            x = np.append(x, np.array([[int(x0), int(x1)]]), axis=0)
        #print(x, y)
    return x[1:], y[1:]
    

In [3]:
x, y = read_data("F1.txt")
print(x, y)

[[   50   -50]
 [  150   -50]
 [  250   -50]
 ...
 [ 3750 -3950]
 [ 3850 -3950]
 [ 3950 -3950]] [0.00900666 0.00914632 0.00937236 ... 0.0150257  0.0125043  0.0105013 ]


In [4]:
# Упаковка данных
def pack_data(x, y):
    x0_max = x[0][0]
    x1_max = x[0][1]
    x0_min = x[0][0]
    x1_min = x[0][1]
    for i in x:
        if i[0] > x0_max: x0_max = i[0]
        if i[1] > x1_max: x1_max = i[1]
        if i[0] < x0_min: x0_min = i[0]
        if i[1] < x1_min: x1_min = i[1]
    #print(x0_max, x1_max)
    #print(x0_min, x1_min)

    x0_delta = x0_max - x0_min
    x1_delta = x1_max - x1_min

    #print(x0_delta)
    #print(x1_delta)

    arr = np.full((x0_delta+1, x1_delta+1), None, dtype=tuple)
    for i in range(x.shape[0]):
        x0 = x[i][0] - x0_min
        x1 = x[i][1] - x1_min
        arr[x0][x1] = ((x[i][0], x[i][1]), y[i])
    
    x0 = 0
    x1 = 0
    for i in arr:
        x1=0
        for k in i:
            if (type(k) is tuple):
                #arr_out[x0][x1] = k
                #print(x0, x1, k)
                x1+=1
        if (type(i[0]) is tuple):
            x0+=1
    arr_out = np.full((x0, x1), None, dtype=tuple)

    x0 = 0
    x1 = 0
    for i in arr:
        x1=0
        for k in i:
            if (type(k) is tuple):
                arr_out[x0][x1] = k
                x1+=1
        if (type(i[0]) is tuple):
            x0+=1

    return arr_out

In [5]:
data = pack_data(x, y)
print(data)

[[((50, -3950), 0.00530738) ((50, -3850), 0.00599818)
  ((50, -3750), 0.00693153) ... ((50, -250), 0.00797303)
  ((50, -150), 0.00862532) ((50, -50), 0.00900666)]
 [((150, -3950), 0.00424952) ((150, -3850), 0.00486655)
  ((150, -3750), 0.0053593) ... ((150, -250), 0.00810639)
  ((150, -150), 0.00894689) ((150, -50), 0.00914632)]
 [((250, -3950), 0.00393672) ((250, -3850), 0.00404523)
  ((250, -3750), 0.00432438) ... ((250, -250), 0.00817544)
  ((250, -150), 0.00877617) ((250, -50), 0.00937236)]
 ...
 [((3750, -3950), 0.0150257) ((3750, -3850), 0.0135209)
  ((3750, -3750), 0.0122265) ... ((3750, -250), 0.0125051)
  ((3750, -150), 0.0129466) ((3750, -50), 0.0133999)]
 [((3850, -3950), 0.0125043) ((3850, -3850), 0.0111755)
  ((3850, -3750), 0.00994691) ... ((3850, -250), 0.0126789)
  ((3850, -150), 0.0130784) ((3850, -50), 0.0133687)]
 [((3950, -3950), 0.0105013) ((3950, -3850), 0.00894342)
  ((3950, -3750), 0.00786142) ... ((3950, -250), 0.0131039)
  ((3950, -150), 0.0136692) ((3950, -50

## Метод перебора

In [6]:
def brute_force(data):
    n = 0
    x_last = data[0][0][0]
    y_last = data[0][0][1]

    for i in data:
        for k in i:
            n += 1
            if (k[1] > y_last):
                y_last = k[1]
                x_last = k[0]

    return x_last, y_last, n

In [7]:
print("Перебор\n")
x, y = read_data("F1.txt")
data = pack_data(x, y)
x_max, y_max, count = brute_force(data)
print("(2)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")
x, y = read_data("F2.txt")
data = pack_data(x, y)
x_max, y_max, count = brute_force(data)
print("(6)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")

Перебор

(2)
x_max = (2050, -1250) 
f(x_max) 0.103408 
Иттераций = 1600 

(6)
x_max = (2650, -1750) 
f(x_max) 0.158663 
Иттераций = 1600 



## Метод поиска по шаблону

In [8]:
def pattern_find(data, pattern, scale, debug=0):
    local_scale = scale
    n = 0
    gamma = 0.7

    ind_last = np.array([data.shape[0]/2, data.shape[1]/2,], dtype=int)

    while True:

        n += 1

        y_last = data[ind_last[0]][ind_last[1]][1]
        ind = ind_last

        for i in pattern:
            ind_tmp = ind_last + np.round(local_scale * i)
            ind_tmp = ind_tmp.astype(int)
            
            # search border
            if(ind_tmp[0] < 0): ind_tmp[0] = 0
            if(ind_tmp[0] >= data.shape[0]): ind_tmp[0] = data.shape[0] - 1
            if(ind_tmp[1] < 0): ind_tmp[1] = 0
            if(ind_tmp[1] >= data.shape[1]): ind_tmp[1] = data.shape[1] - 1

            if (data[ind_tmp[0]][ind_tmp[1]][1] > y_last):
                ind = ind_tmp


        if (all(ind == ind_last)):
            local_scale *= gamma
        else:
            ind_last = ind

        if (local_scale < 1):
            break


    if (debug):
        print(ind_last)
    return data[ind_last[0]][ind_last[1]][0], data[ind_last[0]][ind_last[1]][1], n

In [9]:
scale = 20
pattern = np.array([[0, 0], [0, 1], [1, 0], [-1, 0], [0, -1]])

print("Поиск по шаблону\n")
x, y = read_data("F1.txt")
data = pack_data(x, y)
x_max, y_max, count = pattern_find(data, pattern, scale, debug=1)
print("(2)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")
x, y = read_data("F2.txt")
data = pack_data(x, y)
x_max, y_max, count = pattern_find(data, pattern, scale, debug=1)
print("(6)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")

Поиск по шаблону

[20 27]
(2)
x_max = (2050, -1250) 
f(x_max) 0.103408 
Иттераций = 13 

[26 22]
(6)
x_max = (2650, -1750) 
f(x_max) 0.158663 
Иттераций = 14 



## Метод случайного поиска

In [10]:
def random_find(data, alfa, gamma, M, debug=0):
    n = 0

    ind_last = np.array([data.shape[0]/2, data.shape[1]/2,], dtype=int)
    j = 1
    ksi = np.random.rand(ind_last.shape[0])
    ksi = np.array([i-0.5 for i in ksi])

    while True:

        ind = ind_last + np.round(alfa*ksi)
        ind = ind.astype(int)

        # search border
        if(ind[0] < 0): ind[0] = 0
        if(ind[0] >= data.shape[0]): ind[0] = data.shape[0] - 1
        if(ind[1] < 0): ind[1] = 0
        if(ind[1] >= data.shape[1]): ind[1] = data.shape[1] - 1

        y_last = data[ind_last[0]][ind_last[1]][1]
        y = data[ind[0]][ind[1]][1]

        if (y > y_last):
            ind_last = ind
        else:
            j += 1
            ksi = np.random.rand(ind_last.shape[0])
            ksi = np.array([i-0.5 for i in ksi])

        if (j >= M):
            if (alfa < 1):
                break
            else:
                alfa *= gamma
                j = 1
                
        n += 1
    if (debug):
        print(ind_last)
    return data[ind_last[0]][ind_last[1]][0], data[ind_last[0]][ind_last[1]][1], n

In [11]:
alfa = 60
gamma = 0.75
M = 6
print("Случайного поиска\n")
x, y = read_data("F1.txt")
data = pack_data(x, y)
x_max, y_max, count = random_find(data, alfa, gamma, M, debug=1)
print("(2)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")
x, y = read_data("F2.txt")
data = pack_data(x, y)
x_max, y_max, count = random_find(data, alfa, gamma, M, debug=1)
print("(6)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")

Случайного поиска

[20 27]
(2)
x_max = (2050, -1250) 
f(x_max) 0.103408 
Иттераций = 84 

[26 22]
(6)
x_max = (2650, -1750) 
f(x_max) 0.158663 
Иттераций = 86 



## Метод иммитации отжига

In [12]:
def simulated_annealing(data, t0, t_min, N=10000, debug = 0):

    # Новое значение температуры
    temperature = lambda k: t0 / math.log(1. + k)

    # Вероятность принятия нового состояния
    passage = lambda y_old, y_new, t: math.exp(-1. * (y_new - y_old) / t)

    # Новая начальная точка
    neighbour = lambda ind_old, t: ind_old + np.round(t * np.random.standard_normal(2))

    k = 1

    ind_max = np.array([data.shape[0]/2, data.shape[1]/2,], dtype=int)
    ind = np.array([data.shape[0]/2, data.shape[1]/2,], dtype=int)

    while k < N:
        t = temperature(k)
        ind_new = neighbour(ind, t)
        ind_new = ind_new.astype(int)
        
        # search border
        if(ind_new[0] < 0): ind_new[0] = 0
        if(ind_new[0] >= data.shape[0]): ind_new[0] = data.shape[0] - 1
        if(ind_new[1] < 0): ind_new[1] = 0
        if(ind_new[1] >= data.shape[1]): ind_new[1] = data.shape[1] - 1

        
        y_old = data[ind[0]][ind[1]][1]
        y_new = data[ind_new[0]][ind_new[1]][1]
        y_max = data[ind_max[0]][ind_max[1]][1]

        if y_new > y_old or passage(y_old, y_new, t) >= np.random.rand():
            ind = ind_new

        if y_new > y_max:
            ind_max = ind_new

        k += 1
        if (t < t_min):
            break

    if data[ind_max[0]][ind_max[1]][1] > y_max:
        ind_max = ind
    
    if (debug):
        print(ind_max, t)
    return data[ind_max[0]][ind_max[1]][0], data[ind_max[0]][ind_max[1]][1], k

            

In [13]:
print("Иммитация отжига\n")
x, y = read_data("F1.txt")
data = pack_data(x, y)
x_max, y_max, count = simulated_annealing(data, 40, 6, debug=1)
print("(2)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")
x, y = read_data("F2.txt")
data = pack_data(x, y)
x_max, y_max, count = simulated_annealing(data, 40, 6, debug=1)
print("(6)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")

Иммитация отжига

[20 27] 5.999738898176569
(2)
x_max = (2050, -1250) 
f(x_max) 0.103408 
Иттераций = 786 

[26 22] 5.999738898176569
(6)
x_max = (2650, -1750) 
f(x_max) 0.158663 
Иттераций = 786 



## Генетический алгоритм

In [14]:
def genPoint(old_points, dimSize = 40):
    point = None

    while True:
        point = (np.random.randint(0,dimSize),np.random.randint(0,dimSize))
        if point in old_points:
            continue
        break
    return point

def genetic(data, Iter, N, survival_rate, cross_rate, debug = 0):
    n = 0

    ind_old = [(data.shape[0]//2, data.shape[1]//2)]

    # Generate first generation
    for i in range(N - 1):
        ind_old.append(genPoint(ind_old, data.shape[0]))
    ind_old.sort(key = lambda x: data[x[0]][x[1]][1], reverse=True)

    while True:

        # Get best
        ind_old = ind_old[:int(len(ind_old)*survival_rate)]
        ind = ind_old.copy()

        # Mutation
        fail_gen = 0
        while len(ind) < int(N*cross_rate):
            if (fail_gen > 10):
                break

            crossPoint = None
            parent1,parent2 = random.sample(ind_old, 2)
            crossPoint = (parent1[0],parent2[1])

            # Create uniq pair 
            if crossPoint in ind:
                fail_gen += 1
                continue

            ind.append(crossPoint)


        # Add some new points
        while len(ind) < int(N):
            ind.append(genPoint(ind,data.shape[0]))
        ind.sort(key = lambda x: data[x[0]][x[1]][1], reverse=True)
        
        n+=1
        
        ind_old = ind

        if (n > Iter):
            break

    if (debug):
        print(ind_old[0])
    return data[ind_old[0][0]][ind_old[0][1]][0], data[ind_old[0][0]][ind_old[0][1]][1], n

In [15]:
Iter = 60 # Кол-во шагов
N = 12 # Размер популяции
survival_rate = 0.5 # процент отбираемых точек
cross_rate = 0.8 # процент точек для скрещивания

print("Генетический алгоритм\n")
x, y = read_data("F1.txt")
data = pack_data(x, y)
x_max, y_max, count = genetic(data, Iter, N, survival_rate, cross_rate, debug=1)
print("(2)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")
x, y = read_data("F2.txt")
data = pack_data(x, y)
x_max, y_max, count = genetic(data, Iter, N, survival_rate, cross_rate, debug=1)
print("(6)\nx_max =", x_max, "\nf(x_max)", y_max, "\nИттераций =", count, "\n")

Генетический алгоритм

(20, 25)
(2)
x_max = (2050, -1450) 
f(x_max) 0.09986 
Иттераций = 61 

(26, 21)
(6)
x_max = (2650, -1850) 
f(x_max) 0.148121 
Иттераций = 61 



In [16]:
# Задание 1
#2. Протестировать работу реализованных методов на примере функции п.2.
# Провести исследование влияния настроечных параметров алгоритмов на качество получаемых результатов. 
repeat = 3
for i in range(repeat):

    x, y = read_data("F1.txt")
    data = pack_data(x, y)

    t = PrettyTable(['Метод', 'x_max', 'y', 'Число иттераций', 'Параметры'])

    x_max, y_max, count = brute_force(data)
    t.add_row(['Перебор', x_max, y_max, count, "-"])

    scale = 6
    pattern = np.array([[0, 0], [0, 1], [1, 0], [-1, 0], [0, -1]])
    x_max, y_max, count = pattern_find(data, pattern, scale)
    t.add_row(['Поиск по шаблону', x_max, y_max, count, "scale={}".format(scale)])
    scale = 2
    x_max, y_max, count = pattern_find(data, pattern, scale)
    t.add_row(['Поиск по шаблону', x_max, y_max, count, "scale={}".format(scale)])
    scale = 1
    x_max, y_max, count = pattern_find(data, pattern, scale)
    t.add_row(['Поиск по шаблону', x_max, y_max, count, "scale={}".format(scale)])

    alfa = 10
    gamma = 0.75
    M = 6
    x_max, y_max, count = random_find(data, alfa, gamma, M)
    t.add_row(['Случайный поиск', x_max, y_max, count, "alfa={}, gamma={}, M={}".format(alfa, gamma, M)])
    alfa = 5
    gamma = 0.75
    M = 6
    x_max, y_max, count = random_find(data, alfa, gamma, M)
    t.add_row(['Случайный поиск', x_max, y_max, count, "alfa={}, gamma={}, M={}".format(alfa, gamma, M)])
    alfa = 5
    gamma = 0.7
    M = 6
    x_max, y_max, count = random_find(data, alfa, gamma, M)
    t.add_row(['Случайный поиск', x_max, y_max, count, "alfa={}, gamma={}, M={}".format(alfa, gamma, M)])

    t0 = 30
    t_min = 6
    x_max, y_max, count = simulated_annealing(data, t0, t_min)
    t.add_row(['Иммитация отжига', x_max, y_max, count, "t0={}, t_min={}".format(t0, t_min)])
    t0 = 40
    t_min = 6
    x_max, y_max, count = simulated_annealing(data, t0, t_min)
    t.add_row(['Иммитация отжига', x_max, y_max, count, "t0={}, t_min={}".format(t0, t_min)])
    t0 = 50
    t_min = 6
    x_max, y_max, count = simulated_annealing(data, t0, t_min)
    t.add_row(['Иммитация отжига', x_max, y_max, count, "t0={}, t_min={}".format(t0, t_min)])

    Iter = 60 # Кол-во шагов
    N = 8 # Размер популяции
    survival_rate = 0.5 # процент отбираемых точек
    cross_rate = 0.8 # процент точек для скрещивания
    x_max, y_max, count = genetic(data, Iter, N, survival_rate, cross_rate)
    t.add_row(['Генетический алгоритм', x_max, y_max, count, "Iter={}, N={}, survival_rate={}, cross_rate={}".format(Iter, N, survival_rate, cross_rate)])
    Iter = 60 # Кол-во шагов
    N = 12 # Размер популяции
    survival_rate = 0.5 # процент отбираемых точек
    cross_rate = 0.8 # процент точек для скрещивани
    x_max, y_max, count = genetic(data, Iter, N, survival_rate, cross_rate)
    t.add_row(['Генетический алгоритм', x_max, y_max, count, "Iter={}, N={}, survival_rate={}, cross_rate={}".format(Iter, N, survival_rate, cross_rate)])
    Iter = 100 # Кол-во шагов
    N = 8 # Размер популяции
    survival_rate = 0.5 # процент отбираемых точек
    cross_rate = 0.8 # процент точек для скрещивани
    x_max, y_max, count = genetic(data, Iter, N, survival_rate, cross_rate)
    t.add_row(['Генетический алгоритм', x_max, y_max, count, "Iter={}, N={}, survival_rate={}, cross_rate={}".format(Iter, N, survival_rate, cross_rate)])


    print(t)

+-----------------------+---------------+-----------+-----------------+--------------------------------------------------+
|         Метод         |     x_max     |     y     | Число иттераций |                    Параметры                     |
+-----------------------+---------------+-----------+-----------------+--------------------------------------------------+
|        Перебор        | (2050, -1250) |  0.103408 |       1600      |                        -                         |
|    Поиск по шаблону   | (2050, -1250) |  0.103408 |        9        |                     scale=6                      |
|    Поиск по шаблону   | (2050, -1250) |  0.103408 |        7        |                     scale=2                      |
|    Поиск по шаблону   | (2050, -1250) |  0.103408 |        8        |                     scale=1                      |
|    Случайный поиск    | (2050, -1250) |  0.103408 |        54       |             alfa=10, gamma=0.75, M=6             |
|    Случайный п

In [17]:
# Задание 2
#3. Для функции п.2 подобрать оптимальные по критерию «качество-скорость» параметры настройки каждого алгоритма.

repeat = 1
for i in range(repeat):
    x, y = read_data("F1.txt")
    data = pack_data(x, y)

    t = PrettyTable(['Метод', 'x_max', 'y', 'Число иттераций', 'Параметры'])

    x_max, y_max, count = brute_force(data)
    t.add_row(['Перебор', x_max, y_max, count, "-"])

    scale = 2
    pattern = np.array([[0, 0], [0, 1], [1, 0], [-1, 0], [0, -1]])
    x_max, y_max, count = pattern_find(data, pattern, scale)
    t.add_row(['Поиск по шаблону', x_max, y_max, count, "scale={}".format(scale)])

    alfa = 5
    gamma = 0.7
    M = 6
    x_max, y_max, count = random_find(data, alfa, gamma, M)
    t.add_row(['Случайный поиск', x_max, y_max, count, "alfa={}, gamma={}, M={}".format(alfa, gamma, M)])

    t0 = 40
    t_min = 8
    x_max, y_max, count = simulated_annealing(data, t0, t_min)
    t.add_row(['Иммитация отжига', x_max, y_max, count, "t0={}, t_min={}".format(t0, t_min)])

    Iter = 100 # Кол-во шагов
    N = 8 # Размер популяции
    survival_rate = 0.5 # процент отбираемых точек
    cross_rate = 0.8 # процент точек для скрещивания
    x_max, y_max, count = genetic(data, Iter, N, survival_rate, cross_rate)
    t.add_row(['Генетический алгоритм', x_max, y_max, count, "Iter={}, N={}, survival_rate={}, cross_rate={}".format(Iter, N, survival_rate, cross_rate)])

    print(t)

+-----------------------+---------------+-----------+-----------------+--------------------------------------------------+
|         Метод         |     x_max     |     y     | Число иттераций |                    Параметры                     |
+-----------------------+---------------+-----------+-----------------+--------------------------------------------------+
|        Перебор        | (2050, -1250) |  0.103408 |       1600      |                        -                         |
|    Поиск по шаблону   | (2050, -1250) |  0.103408 |        7        |                     scale=2                      |
|    Случайный поиск    | (2050, -1150) |  0.102552 |        33       |              alfa=5, gamma=0.7, M=6              |
|    Иммитация отжига   | (2050, -1950) | 0.0802925 |       149       |                  t0=40, t_min=8                  |
| Генетический алгоритм | (2050, -1250) |  0.103408 |       101       | Iter=100, N=8, survival_rate=0.5, cross_rate=0.8 |
+---------------

In [18]:
# Задание 3
#5. Сравнить эффективность методов при выбранных параметрах для задачи п.3 и тестовой функции п.4. 
repeat = 1
for i in range(repeat):
    x, y = read_data("F1.txt")
    data_2 = pack_data(x, y)
    x, y = read_data("F2.txt")
    data_4 = pack_data(x, y)

    t = PrettyTable(['Функция', 'Метод', 'x_max', 'y', 'Число иттераций', 'Параметры'])

    x_max, y_max, count = brute_force(data_2)
    t.add_row(['Функция_П2','Перебор', x_max, y_max, count, "-"])
    x_max, y_max, count = brute_force(data_4)
    t.add_row(['Функция_П4','Перебор', x_max, y_max, count, "-"])

    scale = 2
    pattern = np.array([[0, 0], [0, 1], [1, 0], [-1, 0], [0, -1]])
    x_max, y_max, count = pattern_find(data_2, pattern, scale)
    t.add_row(['Функция_П2', 'Поиск по шаблону', x_max, y_max, count, "scale={}".format(scale)])
    x_max, y_max, count = pattern_find(data_4, pattern, scale)
    t.add_row(['Функция_П4', 'Поиск по шаблону', x_max, y_max, count, "scale={}".format(scale)])

    alfa = 5
    gamma = 0.7
    M = 6
    x_max, y_max, count = random_find(data_2, alfa, gamma, M)
    t.add_row(['Функция_П2', 'Случайный поиск', x_max, y_max, count, "alfa={}, gamma={}, M={}".format(alfa, gamma, M)])
    x_max, y_max, count = random_find(data_4, alfa, gamma, M)
    t.add_row(['Функция_П4', 'Случайный поиск', x_max, y_max, count, "alfa={}, gamma={}, M={}".format(alfa, gamma, M)])

    t0 = 40
    t_min = 8
    x_max, y_max, count = simulated_annealing(data_2, t0, t_min)
    t.add_row(['Функция_П2', 'Иммитация отжига', x_max, y_max, count, "t0={}, t_min={}".format(t0, t_min)])
    x_max, y_max, count = simulated_annealing(data_4, t0, t_min)
    t.add_row(['Функция_П4', 'Иммитация отжига', x_max, y_max, count, "t0={}, t_min={}".format(t0, t_min)])

    Iter = 100 # Кол-во шагов
    N = 8 # Размер популяции
    survival_rate = 0.5 # процент отбираемых точек
    cross_rate = 0.8 # процент точек для скрещивания
    x_max, y_max, count = genetic(data_2, Iter, N, survival_rate, cross_rate)
    t.add_row(['Функция_П2', 'Генетический алгоритм', x_max, y_max, count, "Iter={}, N={}, survival_rate={}, cross_rate={}".format(Iter, N, survival_rate, cross_rate)])
    x_max, y_max, count = genetic(data_4, Iter, N, survival_rate, cross_rate)
    t.add_row(['Функция_П4', 'Генетический алгоритм', x_max, y_max, count, "Iter={}, N={}, survival_rate={}, cross_rate={}".format(Iter, N, survival_rate, cross_rate)])

    print(t)


+------------+-----------------------+---------------+-----------+-----------------+--------------------------------------------------+
|  Функция   |         Метод         |     x_max     |     y     | Число иттераций |                    Параметры                     |
+------------+-----------------------+---------------+-----------+-----------------+--------------------------------------------------+
| Функция_П2 |        Перебор        | (2050, -1250) |  0.103408 |       1600      |                        -                         |
| Функция_П4 |        Перебор        | (2650, -1750) |  0.158663 |       1600      |                        -                         |
| Функция_П2 |    Поиск по шаблону   | (2050, -1250) |  0.103408 |        7        |                     scale=2                      |
| Функция_П4 |    Поиск по шаблону   | (1950, -1750) | 0.0726873 |        5        |                     scale=2                      |
| Функция_П2 |    Случайный поиск    | (2050, -1